In [1]:
import sys
sys.path.append("../../")
import os
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.io.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Tensorflow version: 1.15.2


In [2]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
#EPOCHS_FOR_CRITEO_RUN = 30
BATCH_SIZE_SYNTHETIC = 128
#BATCH_SIZE_CRITEO = 4096
RANDOM_SEED = SEED  # Set None for non-deterministic result

In [3]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = r'foodfmtrain'
valid_file =  r'foodfmvalid'
test_file = r'foodfmtest'
predictfile = r'foodTest'

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


100%|██████████| 10.3k/10.3k [00:01<00:00, 7.46kKB/s]


In [4]:
yaml_file

'/var/folders/gd/f1pg0x2s5wsf5jymdrnz61nh0000gn/T/tmpgygviacl/xDeepFM.yaml'

In [5]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1030, 
                          FIELD_COUNT=2, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

kg_file=None,user_clicks=None,FEATURE_COUNT=1030,FIELD_COUNT=2,data_format=ffm,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=None,entityEmb_file=None,contextEmb_file=None,news_feature_file=None,user_history_file=None,use_context=None,doc_size=None,word_size=None,entity_size=None,entity_dim=None,entity_embedding_method=None,transform=None,train_ratio=None,dim=10,layer_sizes=[100, 100],cross_layer_sizes=[1],cross_layers=None,activation=['relu', 'relu'],cross_activation=identity,user_dropout=False,dropout=[0.0, 0.0],attent

In [6]:
#input_creator = FFMTextIterator


In [6]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 
model.fit(train_file, valid_file)
# check the predictive performance after the model is trained
res_real = model.run_eval(test_file)
print(res_real)
pm.record("res_real", res_real)





Add CIN part.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where











{'auc': 0.509, 'logloss': 0.693}
at epoch 1
train info: logloss loss:0.7199932344352142
eval info: auc:0.4929, logloss:0.6878
at epoch 1 , train time: 0.4 eval time: 0.4
at epoch 2
train info: logloss loss:0.7061250006096272
eval info: auc:0.4936, logloss:0.6838
at epoch 2 , train time: 0.2 eval time: 0.3
at epoch 3
train info: logloss loss:0.6961172713509088
eval info: auc:0.4944, logloss:0.6811
at epoch 3 , train time: 0.2 eval time: 0.3
at epoch 4
train info: logloss loss:0.6880952225455755
eval info: auc:0.4956, logloss:0.6795
at epoch 4 , train time: 0.2 eval time: 0.3
at epoch 5
train info: logloss loss:0.6794489071338992
eval info: auc:0.4969, logloss:0.6793
at epoch 5 , train time: 0.3 eval time: 0.3
at epoch 6
train info: logloss loss:0.6664385290085515
eval info: auc:0.4987, logloss:0.6813
at epoch 6 , train time: 0.3 eval time: 0.3
at epoch 7
train i

In [7]:
model.predict(predictfile, "fmfoodlargetestfile.txt")
?model.predict

Signature: model.predict(infile_name, outfile_name)
Docstring:
Make predictions on the given data, and output predicted scores to a file.

Args:
    infile_name (str): Input file name, format is same as train/val/test file.
    outfile_name (str): Output file name, each line is the predict score.

Returns:
    obj: An instance of self.
File:      ~/ADM/Assignment3-Recommender System/reco_utils/recommender/deeprec/models/base_model.py
Type:      method


In [9]:
# import pickle
# pickle.dump(model, open("abc.pickle", 'wb'))

In [10]:
# type(model)

In [11]:
# import joblib
# import pickle
# joblib.dump(model, 'model.joblib')

In [8]:
import pandas as pd


In [9]:
prediction_df = pd.read_fwf('fmfoodlargetestfile.txt',header=None)


In [18]:
prediction_df.columns = ["Like Probability"]

In [19]:
prediction_df

,Like Probability
0,0.271233
1,0.022452
2,0.011625
3,0.057317
4,0.121535
...,...
29995,0.614679
29996,0.228405
29997,0.499540
29998,0.315276


In [14]:
prediction_df1 = prediction_df.copy()

In [15]:
prediction_df1['Like Probability'] = prediction_df1['Like Probability'].round(decimals=3)

In [16]:
prediction_df1

,Like Probability
0,0.271
1,0.022
2,0.012
3,0.057
4,0.122
...,...
29995,0.615
29996,0.228
29997,0.500
29998,0.315


In [20]:
data_df = pd.read_csv("datatopredict.csv")
data_df

,Unnamed: 0,userId,productId,ratings
0,0,411,115,1
1,1,244,122,0
2,2,755,102,0
3,3,391,126,0
4,4,778,109,1
...,...,...,...,...
29995,29995,367,109,0
29996,29996,524,129,0
29997,29997,794,101,0
29998,29998,314,116,1


In [21]:
data_df1= data_df[["userId","productId"]]
data_df1

,userId,productId
0,411,115
1,244,122
2,755,102
3,391,126
4,778,109
...,...,...
29995,367,109
29996,524,129
29997,794,101
29998,314,116


In [22]:
result = pd.concat([data_df1, prediction_df1], axis=1, join='inner')


In [23]:
result

,userId,productId,Like Probability
0,411,115,0.271
1,244,122,0.022
2,755,102,0.012
3,391,126,0.057
4,778,109,0.122
...,...,...,...
29995,367,109,0.615
29996,524,129,0.228
29997,794,101,0.500
29998,314,116,0.315


In [24]:
result2 = result.copy()

In [25]:
food_df = pd.read_csv("FoodDetails.csv")

In [26]:
food_df

,productId,Product Name,Price
0,116,Salad,6
1,121,Milk,9
2,126,Tuna,5
3,109,Cupcake,9
4,124,Carrots,10
5,105,Meat,15
6,110,Curly Kale,5
7,111,Cereals,5
8,101,Bread,11
9,100,Juice,6


In [27]:
img = ['https://adm-food-probability.s3.amazonaws.com/salad.jpg',
      'https://adm-food-probability.s3.amazonaws.com/milk.jpg',
      'https://adm-food-probability.s3.amazonaws.com/tuna.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cupcakes.jpg',
      'https://adm-food-probability.s3.amazonaws.com/carrots.jpg',
       'https://adm-food-probability.s3.amazonaws.com/meat.jpg',
      'https://adm-food-probability.s3.amazonaws.com/curlykale.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cereal.jpg',
      'https://adm-food-probability.s3.amazonaws.com/bread.jpg',
      'https://adm-food-probability.s3.amazonaws.com/juice.jpg',
      'https://adm-food-probability.s3.amazonaws.com/banana.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cheese.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cookie.jpg',
      'https://adm-food-probability.s3.amazonaws.com/curd.jpg',
      'https://adm-food-probability.s3.amazonaws.com/grapes.jpg',
      'https://adm-food-probability.s3.amazonaws.com/eggs.jpg',
      'https://adm-food-probability.s3.amazonaws.com/avocado.jpg',
      'https://adm-food-probability.s3.amazonaws.com/chips.jpg',
      'https://adm-food-probability.s3.amazonaws.com/coconut water.jpg',
      'https://adm-food-probability.s3.amazonaws.com/zuccini.jpg',
      'https://adm-food-probability.s3.amazonaws.com/walnut.jpg',
      'https://adm-food-probability.s3.amazonaws.com/pizza.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cabbage.jpg',
      'https://adm-food-probability.s3.amazonaws.com/fish.jpg',
      'https://adm-food-probability.s3.amazonaws.com/cereals.jpg',
      'https://adm-food-probability.s3.amazonaws.com/spaghetti.jpg',
      'https://adm-food-probability.s3.amazonaws.com/rice.jpg',
      'https://adm-food-probability.s3.amazonaws.com/apple.jpg',
      'https://adm-food-probability.s3.amazonaws.com/tortilla.jpg',
      'https://adm-food-probability.s3.amazonaws.com/frozen.jpg']

In [30]:
food_df['img']=img

In [31]:
xdeepfmfinal = pd.merge(result2,food_df, on='productId')

In [33]:
xdeepfmfinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29000 entries, 0 to 28999
Data columns (total 6 columns):
userId              29000 non-null int64
productId           29000 non-null int64
Like Probability    29000 non-null float64
Product Name        29000 non-null object
Price               29000 non-null int64
img                 29000 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 1.5+ MB


In [34]:
xdeepfmfinal["userId"] = xdeepfmfinal["userId"].astype(str)
xdeepfmfinal["productId"] = xdeepfmfinal["productId"].astype(str)

In [35]:
xdeepfmfinal = xdeepfmfinal.sort_values(['Like Probability'], ascending=False)

In [36]:
xdeepfmfinal

,userId,productId,Like Probability,Product Name,Price,img
4265,91,109,1.000,Cupcake,9,https://adm-food-probability.s3.amazonaws.com/...
1287,124,122,1.000,Tortilla,6,https://adm-food-probability.s3.amazonaws.com/...
2724,991,102,0.999,Sphagetti,14,https://adm-food-probability.s3.amazonaws.com/...
13867,991,105,0.998,Meat,15,https://adm-food-probability.s3.amazonaws.com/...
15969,884,104,0.998,Cookie,5,https://adm-food-probability.s3.amazonaws.com/...
...,...,...,...,...,...,...
2236,859,102,0.001,Sphagetti,14,https://adm-food-probability.s3.amazonaws.com/...
6715,872,108,0.001,Grapes,7,https://adm-food-probability.s3.amazonaws.com/...
4286,884,109,0.000,Cupcake,9,https://adm-food-probability.s3.amazonaws.com/...
3440,902,126,0.000,Tuna,5,https://adm-food-probability.s3.amazonaws.com/...


In [37]:
(xdeepfmfinal[xdeepfmfinal["userId"] == "2"]).sort_values(by='Like Probability', ascending=False).reset_index(drop=True).loc[0:4,:]

,userId,productId,Like Probability,Product Name,Price,img
0,2,105,0.931,Meat,15,https://adm-food-probability.s3.amazonaws.com/...
1,2,115,0.929,Cabbage,5,https://adm-food-probability.s3.amazonaws.com/...
2,2,126,0.918,Tuna,5,https://adm-food-probability.s3.amazonaws.com/...
3,2,120,0.877,Eggs,8,https://adm-food-probability.s3.amazonaws.com/...
4,2,102,0.766,Sphagetti,14,https://adm-food-probability.s3.amazonaws.com/...


In [ ]:
import pickle
pickle.dump(xdeepfmfinal, open("xdeepfmfinalimage.pkl", 'wb'))